# Apply Vegging to ResNet-18: CIFAR-10 Dataset

### Preparation

In [5]:
# Import modules
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

### Set GPU

In [6]:
# Check available device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Load Dataset and Pre-Processing

In [8]:
# Set transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize dataset between '-1' and '1'
])

In [9]:
# Load datasets
train_dataset = CIFAR10(root = './data/0707-CIFAR10-ResNet18', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root = './data/0707-CIFAR10-ResNet18', train=True, download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/0707-CIFAR10-ResNet18\cifar-10-python.tar.gz to ./data/0707-CIFAR10-ResNet18
Files already downloaded and verified


In [12]:
# Set DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

print(train_loader)
print(test_loader)

### Define Models

In [15]:
# Define a model
model = resnet18(pretrained=True)
print(model)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\genih/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
# Adjust parameters
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)

In [18]:
# Define a model for Ensemble
bagging_model = BaggingClassifier(
    base_estimator = DecisionTreeClassifier(max_depth = 7),
    n_estimators = 5
)
print(bagging_model)

BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=7),
                  n_estimators=5)


### Set Loss Function and Optimizer

In [19]:
# Set Loss Function
criterion = nn.CrossEntropyLoss()

# Set Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Define Learning Models

In [24]:
# Define a trainning model
def train(mdel, device, train_loader, optimizer, criterion):
    # Set a mode
    model.train()
    
    # Fit models
    for batch_idex, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Initialzie optimizer
        optimizer.zero_grad()
        
        # Set output
        output = model(data)
        
        # Set Loss Function
        loss = criterion(output, target)
        
        # Set backpropagation
        loss.backward()
        optimizer.step()
        
# Define an evaluation model
def evaluate(model, device, test_loader):
    # Set a mode
    model.eval()
    
    # Initialize lists
    predictions = []
    targets = []
    
    # Fit models
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            # Set output
            output = model(data)
            
            # Get values of predicted row
            _, predicted = torch.max(output, 1)
            
            # Transform data in a NumPy array format
            predictions.extend(predicted.cpu().numpy())
            targets.extend(target.cpu().numpy())
            
    # Calcualte accuracy
    accuracy = accuracy_score(targets, predictions)
    
    return accuracy


# Define a prediction model with ensemble techniques
def ensemble_predict(models, device, test_loader):
    # Initialize lists
    predictions = []
    
    # Fit models
    with torch.no_grad():
        # Get data
        for data, _ in test_loader:
            data = data.to(device)
            outputs = []
            for model in models:
                model = model.to(device)
                model.eval()
                output = model(data)
                outputs.append(output)
            
            # Set output
            ensemble_output = torch.stack(outputs).mean(dim=0)
            _, predicted = torch.max(ensemble_output, 1)
            predictions.extend(predicted.cpu().numpy())
    
    return predictions

### Run the Model

In [26]:
if __name__ == '__main__':
    # Initialize lists
    models = []
    
    # Fit model
    for epoch in range(1, 6):  # Use 5 models for Bagging Ensemble model
        print(f'Training Model {epoch}')
        
        model = model.to(device)
        train(model, device, train_loader, optimizer, criterion)
        
        accuracy = evaluate(model, device, test_loader)
        print(f'Model {epoch} Accuracy: {accuracy:.2f}')
        models.append(model)
    
    # Get results
    ensemble_predictions = ensemble_predict(models, device, test_loader)
    ensemble_accuracy = accuracy_score(test_dataset.targets, ensemble_predictions)
    print(f'\nEnsemble Accuracy: {ensemble_accuracy:.2f}')

Training Model 1
Model 1 Accuracy: 0.96
Training Model 2
Model 2 Accuracy: 0.96
Training Model 3
Model 3 Accuracy: 0.97
Training Model 4
Model 4 Accuracy: 0.98
Training Model 5
Model 5 Accuracy: 0.98

Ensemble Accuracy: 0.98
